In [1]:
import pandas as pd
import re
import string

In [2]:
data=pd.read_csv("train.csv")

In [3]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
data=data.drop(['id','author','title'],axis=1)

In [5]:
data

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1
...,...,...
20795,Rapper T. I. unloaded on black celebrities who...,0
20796,When the Green Bay Packers lost to the Washing...,0
20797,The Macy’s of today grew from the union of sev...,0
20798,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [6]:
data.isnull().sum()

text     39
label     0
dtype: int64

In [7]:
data=data.fillna('')
data.isnull().sum()

text     0
label    0
dtype: int64

In [8]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text


In [9]:
x=data.text
x

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        Ever get the feeling your life circles the rou...
2        Why the Truth Might Get You Fired October 29, ...
3        Videos 15 Civilians Killed In Single US Airstr...
4        Print \nAn Iranian woman has been sentenced to...
                               ...                        
20795    Rapper T. I. unloaded on black celebrities who...
20796    When the Green Bay Packers lost to the Washing...
20797    The Macy’s of today grew from the union of sev...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799      David Swanson is an author, activist, journa...
Name: text, Length: 20800, dtype: object

In [10]:

x=x.apply(wordopt)
    

In [11]:
x

0        house dem aide  we didn t even see comey s let...
1        ever get the feeling your life circles the rou...
2        why the truth might get you fired october     ...
3        videos  civilians killed in single us airstrik...
4        print  an iranian woman has been sentenced to ...
                               ...                        
20795    rapper t  i  unloaded on black celebrities who...
20796    when the green bay packers lost to the washing...
20797    the macy s of today grew from the union of sev...
20798    nato  russia to hold parallel exercises in bal...
20799      david swanson is an author  activist  journa...
Name: text, Length: 20800, dtype: object

In [12]:
y=data.label
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()
xt = vectorization.fit_transform(x)
print(xt)

  (0, 146526)	0.04527595344903779
  (0, 82340)	0.043225088051980096
  (0, 18599)	0.021018138520394813
  (0, 58603)	0.012004602982852441
  (0, 24029)	0.023211718501786462
  (0, 44901)	0.017136116887597908
  (0, 26573)	0.0556612279806733
  (0, 93934)	0.00805628699808308
  (0, 31115)	0.04570852606780128
  (0, 48125)	0.014422135752519308
  (0, 88946)	0.03719389794814673
  (0, 146857)	0.00896608060192663
  (0, 80376)	0.025007722454010953
  (0, 146467)	0.011035908650313862
  (0, 131874)	0.013385607628835387
  (0, 675)	0.03270885898476147
  (0, 127899)	0.0273275279596678
  (0, 63433)	0.021476071451701054
  (0, 120349)	0.025898104659937533
  (0, 115754)	0.028533456559008376
  (0, 131823)	0.010741350357451263
  (0, 138396)	0.0089545421179176
  (0, 124686)	0.019025107746837796
  (0, 33730)	0.023757025627560224
  (0, 75647)	0.020617638830697452
  :	:
  (20799, 91471)	0.031899476767720696
  (20799, 59438)	0.01357288121517501
  (20799, 143858)	0.014242111153961914
  (20799, 131232)	0.03994200351233

In [14]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train, y_test = train_test_split(xt , y ,test_size=0.7, stratify = y)

In [15]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

In [16]:
y_pred=model.predict(x_test)

In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred , y_test)

0.9272664835164836

In [18]:
y_pred

array([1, 1, 0, ..., 0, 0, 1], dtype=int64)

In [19]:
import pickle

In [20]:
pickle.dump(model,open('model1.pkl','wb'))
model=pickle.load(open('model1.pkl','rb'))

In [21]:

    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = model.predict(new_xv_test)
    return print(pred_LR)
  

In [22]:
import tkinter as tk
from tkinter import *
import pickle
from tkinter import messagebox
from PIL import ImageTk,Image



In [29]:
win=tk.Tk()
win.title('FAKE NEWS DETECTOR')
win.geometry("500x300")

win.configure(background="light blue")
scroll=Scrollbar(win)
scroll.pack(side=RIGHT,fill=Y)
img = ImageTk.PhotoImage(Image.open("img6.jpg"))

l=Label(win,text="Fake News Detector",width=200)
l.pack()
canvas = Canvas(
	win, 
	width = 500,
	height = 300
	)
canvas.pack(fill='both', expand = True)

canvas.create_image(
	0, 
	0, 
	image=img,
	anchor = "nw"
	)


text = Text(
    win,
    height=10,
    bg="light yellow",
    wrap='word',
    
    width=53,
    yscrollcommand=scroll.set
    
)
text.place(x=30, y=50)


scroll.config(command=text.yview)
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text    
    
def manual_testing():
    news=text.get(0.0,END)
       
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = model.predict(new_xv_test)
    k=pred_LR
    
    if(k==1):
        
        messagebox.showinfo("Real or Fake",'This is Fake News')

        
    elif(k==0):

        messagebox.showinfo("Real or Fake",'This is Real News')
    else:
        
        messagebox.showinfo('real or fake','none')
bt=tk.Button(win,text="Verify",fg="black",bg="white",padx=50,pady=10,command=manual_testing)
bt.place(x=160,y=220)
win.mainloop()


